## Preprocessing

In [23]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [24]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_dropped = application_df.drop(['EIN','NAME'], axis=1)

In [25]:
# Determine the number of unique values in each column.
application_df_dropped.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [26]:
# Look at APPLICATION_TYPE value counts for binning
application_df_dropped.value_counts('APPLICATION_TYPE')

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
dtype: int64

In [27]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
cut_off = 100 

application_counts = application_df_dropped.value_counts('APPLICATION_TYPE')
application_types_to_replace = application_counts[application_counts < cut_off].index.to_list()
# Replace in dataframe
for app in application_types_to_replace:
    application_df_dropped['APPLICATION_TYPE'] = application_df_dropped['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_dropped.value_counts('APPLICATION_TYPE')

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
dtype: int64

In [28]:
# Look at CLASSIFICATION value counts for binning
application_df_dropped.value_counts('CLASSIFICATION')

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2190        1
C2380        1
C2500        1
C2561        1
C8210        1
Length: 71, dtype: int64

In [29]:
# You may find it helpful to look at CLASSIFICATION value counts >1
 
application_df_dropped.value_counts('CLASSIFICATION')[application_df_dropped.value_counts('CLASSIFICATION') > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C1720        6
C4100        6
C2400        6
C1600        5
C1257        5
C2710        3
C1260        3
C0           3
C1267        2
C1246        2
C1256        2
C3200        2
C1234        2
dtype: int64

In [30]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 1 
classification_counts = application_df_dropped.value_counts('CLASSIFICATION')
classifications_to_replace = classification_counts[classification_counts <= cutoff].index.to_list()
# Replace in dataframe
for cls in classifications_to_replace:
    application_df_dropped.loc[application_df_dropped['CLASSIFICATION'] == cls, 'CLASSIFICATION'] = "Other"
    # application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
# Check to make sure binning was successful
new_classification_counts = application_df_dropped.value_counts('CLASSIFICATION')
new_classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Other       26
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
C1235        9
C7210        7
C4100        6
C1720        6
C2400        6
C1600        5
C1257        5
C2710        3
C1260        3
C0           3
C3200        2
C1267        2
C1256        2
C1246        2
C1234        2
dtype: int64

In [31]:
application_df_dropped.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [32]:
# Convert categorical data to numeric with `pd.get_dummies`

categorical_cols = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']
dummies = pd.get_dummies(application_df_dropped[categorical_cols])
application_df_dropped = application_df_dropped.drop(categorical_cols, axis=1)
application_df_dropped = pd.concat([application_df_dropped, dummies], axis=1)
application_df_dropped.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [33]:
# Split our preprocessed data into our features and target arrays
target = application_df_dropped['IS_SUCCESSFUL']
features = application_df_dropped.drop('IS_SUCCESSFUL', axis=1)

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [34]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [35]:
X_train_scaled 

array([[ 0.01350021, -0.03115182, -0.05986953, ..., -0.07135527,
         0.02832705, -0.02832705],
       [ 0.01350021, -0.03115182, -0.05986953, ..., -0.07135527,
         0.02832705, -0.02832705],
       [ 0.01350021, -0.03115182, -0.05986953, ..., -0.07135527,
         0.02832705, -0.02832705],
       ...,
       [ 0.01350021, -0.03115182, -0.05986953, ..., -0.07135527,
         0.02832705, -0.02832705],
       [ 0.01350021, -0.03115182, -0.05986953, ..., -0.07135527,
         0.02832705, -0.02832705],
       [ 0.01350021, -0.03115182, -0.05986953, ..., -0.07135527,
         0.02832705, -0.02832705]])

## Neural Network Model Adjustments
- Attempt 1 : Adding a third hidden layer (get accuracy 0.7271)

In [36]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=40, activation="relu", input_dim=input_features))

# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=20, activation='relu'))


# Adding a third hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation='relu')) 


# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) 
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 40)                3400      
                                                                 
 dense_1 (Dense)             (None, 20)                820       
                                                                 
 dense_2 (Dense)             (None, 10)                210       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 4,441
Trainable params: 4,441
Non-trainable params: 0
_________________________________________________________________


In [37]:
# Compile the model
#  YOUR CODE GOES HERE
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01) 
# nn.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])




In [38]:
# from tensorflow.keras.callbacks import EarlyStopping

# early_stopping = EarlyStopping(monitor='val_loss', patience=10)
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=64)


Epoch 1/100
429/429 [==============================] - 1s 933us/step - loss: 0.5908 - accuracy: 0.7059
Epoch 2/100
429/429 [==============================] - 0s 975us/step - loss: 0.5526 - accuracy: 0.7305
Epoch 3/100
429/429 [==============================] - 0s 925us/step - loss: 0.5480 - accuracy: 0.7314
Epoch 4/100
429/429 [==============================] - 0s 1ms/step - loss: 0.5461 - accuracy: 0.7342
Epoch 5/100
429/429 [==============================] - 0s 951us/step - loss: 0.5446 - accuracy: 0.7330
Epoch 6/100
429/429 [==============================] - 0s 990us/step - loss: 0.5438 - accuracy: 0.7350
Epoch 7/100
429/429 [==============================] - 0s 913us/step - loss: 0.5424 - accuracy: 0.7344
Epoch 8/100
429/429 [==============================] - 0s 892us/step - loss: 0.5423 - accuracy: 0.7346
Epoch 9/100
429/429 [==============================] - 0s 978us/step - loss: 0.5420 - accuracy: 0.7345
Epoch 10/100
429/429 [==============================] - 0s 1ms/step - loss:

In [39]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5656 - accuracy: 0.7271 - 341ms/epoch - 2ms/step
Loss: 0.565573513507843, Accuracy: 0.7271137237548828


- Attempt 2 : Implement dropout to reduce overfitting (get accuracy 0.7303)

In [40]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

input_features = len(X_train_scaled[0])
nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=40, activation="relu", input_dim=input_features))
nn.add(tf.keras.layers.Dropout(0.5)) 

# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=20, activation='relu'))

# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) 
# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 40)                3400      
                                                                 
 dropout (Dropout)           (None, 40)                0         
                                                                 
 dense_5 (Dense)             (None, 20)                820       
                                                                 
 dense_6 (Dense)             (None, 1)                 21        
                                                                 
Total params: 4,241
Trainable params: 4,241
Non-trainable params: 0
_________________________________________________________________


In [41]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [42]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=64)


Epoch 1/100


429/429 [==============================] - 1s 1ms/step - loss: 0.6475 - accuracy: 0.6645
Epoch 2/100
429/429 [==============================] - 0s 1ms/step - loss: 0.5944 - accuracy: 0.7138
Epoch 3/100
429/429 [==============================] - 0s 1ms/step - loss: 0.5809 - accuracy: 0.7198
Epoch 4/100
429/429 [==============================] - 0s 1ms/step - loss: 0.5739 - accuracy: 0.7227
Epoch 5/100
429/429 [==============================] - 1s 1ms/step - loss: 0.5711 - accuracy: 0.7236
Epoch 6/100
429/429 [==============================] - 1s 1ms/step - loss: 0.5679 - accuracy: 0.7258
Epoch 7/100
429/429 [==============================] - 1s 1ms/step - loss: 0.5622 - accuracy: 0.7269
Epoch 8/100
429/429 [==============================] - 0s 1ms/step - loss: 0.5596 - accuracy: 0.7290
Epoch 9/100
429/429 [==============================] - 0s 1ms/step - loss: 0.5612 - accuracy: 0.7289
Epoch 10/100
429/429 [==============================] - 1s 2ms/step - loss: 0.5567 - accuracy: 0.7290
E

In [43]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5571 - accuracy: 0.7303 - 286ms/epoch - 1ms/step
Loss: 0.5571165680885315, Accuracy: 0.7303206920623779


- Attempt 3 : Implement a different optimizer (get accuracy 0.7306)

In [44]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

input_features = len(X_train_scaled[0])
nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=input_features))
nn.add(tf.keras.layers.Dropout(0.3)) 

# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=40, activation='relu'))

# Adding a third hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation='relu')) 

# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) 
# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                6800      
                                                                 
 dropout_1 (Dropout)         (None, 80)                0         
                                                                 
 dense_8 (Dense)             (None, 40)                3240      
                                                                 
 dense_9 (Dense)             (None, 20)                820       
                                                                 
 dense_10 (Dense)            (None, 1)                 21        
                                                                 
Total params: 10,881
Trainable params: 10,881
Non-trainable params: 0
_________________________________________________________________


In [45]:
# nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01) 
nn.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])


In [46]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=64)


Epoch 1/100


429/429 [==============================] - 1s 1ms/step - loss: 0.6926 - accuracy: 0.5617
Epoch 2/100
429/429 [==============================] - 1s 1ms/step - loss: 0.6318 - accuracy: 0.6873
Epoch 3/100
429/429 [==============================] - 1s 1ms/step - loss: 0.5973 - accuracy: 0.7109
Epoch 4/100
429/429 [==============================] - 1s 1ms/step - loss: 0.5876 - accuracy: 0.7164
Epoch 5/100
429/429 [==============================] - 1s 1ms/step - loss: 0.5802 - accuracy: 0.7170
Epoch 6/100
429/429 [==============================] - 1s 1ms/step - loss: 0.5795 - accuracy: 0.7187
Epoch 7/100
429/429 [==============================] - 1s 1ms/step - loss: 0.5729 - accuracy: 0.7218
Epoch 8/100
429/429 [==============================] - 1s 1ms/step - loss: 0.5723 - accuracy: 0.7223
Epoch 9/100
429/429 [==============================] - 1s 1ms/step - loss: 0.5713 - accuracy: 0.7244
Epoch 10/100
429/429 [==============================] - 1s 2ms/step - loss: 0.5699 - accuracy: 0.7233
E

In [47]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5548 - accuracy: 0.7306 - 305ms/epoch - 1ms/step
Loss: 0.5547918677330017, Accuracy: 0.7306122183799744


In [48]:
# Export our model to HDF5 file
nn.save('2_model.h5')  # This will save the model as 'my_model.h5'